In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil

# 1) 專案資料夾（相對於你 .ipynb 的位置）
ROOT  = Path.cwd()
OUT   = ROOT / "outputs"      # 統一輸出資料夾
FIGS  = OUT / "figs"
CACHE = OUT / "cache"
for p in (OUT, FIGS, CACHE):
    p.mkdir(parents=True, exist_ok=True)

def save_fig(fig, name, dpi=300):
    """同時存 PNG/PDF/SVG；一定放在 plt.show() 之前或之後都可，但『要呼叫』"""
    for ext in ("png", "pdf", "svg"):
        fig.savefig(FIGS / f"{name}.{ext}", dpi=dpi, bbox_inches="tight")

def save_cache(t_hr, dT_obs, y_dual, y_ils, params_dual, params_ils, hist=None):
    """存成 .npz（之後可直接載入，不必重算）"""
    np.savez(CACHE / "fit_cache.npz",
             t_hr=t_hr, dT_obs=dT_obs, y_dual=y_dual, y_ils=y_ils,
             params_dual=params_dual, params_ils=params_ils,
             hist=hist if hist is not None else np.array([]))

def load_cache():
    """若有快取就載入；沒有就回傳 None"""
    f = CACHE / "fit_cache.npz"
    if not f.exists(): return None
    z = np.load(f, allow_pickle=True)
    return {k: z[k] for k in z.files}

def export_excel(t_hr, dT_obs, y_dual, y_ils, params_dual, params_ils, hist=None):
    """把數據輸出成一個 Excel（多工作表），老師直接打開就看得懂"""
    xlsx = OUT / "results.xlsx"
    with pd.ExcelWriter(xlsx, engine="openpyxl") as xw:
        pd.DataFrame({
            "t_hr": t_hr, "dT_obs": dT_obs,
            "DualLag": y_dual, "ILS": y_ils
        }).to_excel(xw, sheet_name="fit_series", index=False)

        pd.DataFrame([params_dual]).to_excel(xw, sheet_name="dual_params", index=False)
        pd.DataFrame([params_ils]).to_excel(xw, sheet_name="ils_params", index=False)

        if hist is not None and len(hist):
            df_hist = pd.DataFrame(hist, columns=["tmax_hr","k","C","tau_q","tau_T","q"])
            df_hist.to_excel(xw, sheet_name="truncation", index=False)
    return xlsx

def export_all(fig_fit=None, fig_conv=None,
               t_hr=None, dT_obs=None, y_dual=None, y_ils=None,
               params_dual=None, params_ils=None, hist=None):
    """一鍵輸出：快取 + Excel + 圖檔（可選）"""
    assert t_hr is not None, "export_all: 請把你的結果傳進來"
    save_cache(t_hr, dT_obs, y_dual, y_ils, params_dual, params_ils, hist)
    excel_path = export_excel(t_hr, dT_obs, y_dual, y_ils, params_dual, params_ils, hist)
    if fig_fit is not None:  save_fig(fig_fit,  "fit_dual_vs_ils")
    if fig_conv is not None: save_fig(fig_conv, "convergence")
    print("✅ 已輸出到：", OUT.resolve())

def make_submission_zip():
    """把 outputs 打包成 zip，直接傳給老師"""
    zip_path = shutil.make_archive(str(OUT / "submission"), 'zip', OUT)
    print("📦 已建立壓縮檔：", zip_path)
